In [1]:
import numpy as np
from scipy.special import gamma

In [2]:
# PARAMETERS
Tmax = 14400
nbin = 32
dt = 100. #s
scal = 1 # scaling factor for grid
emin = 7.091336e-10 # mg; minimum mass for grid
rhow = 1e-9 # mg/um^3

nt = int(Tmax/dt)

printtimes = np.asarray([0, Tmax//dt, Tmax/dt],dtype='int')

B = 1500*1e-12 #1/sec

# initial condition
k1 = 3
theta = 100 #um^3
N0 = 100 # cm^-3

In [3]:
# function for initial condition
def gamma_init(x, N, k, theta):
    return N*(x/theta)**k/gamma(k)*np.exp(-x/theta)

# functions for kernel
def golovin_kernel(x1,x2,B):
    return B*(x1+x2)

In [4]:
def courant(vgrid,dlnr):
    nbin = len(vgrid)
    c = np.zeros((nbin,nbin))
    ima = np.zeros((nbin,nbin))
    for i in range(nbin):
        for j in range(i, nbin):
            v0 = vgrid[i] + vgrid[j]
            for k in range(j, nbin):
                if vgrid[k] >= v0 and vgrid[k-1] <= v0:
                    if c[i,j] < 1-1e-8:
                        kk = k-1
                        c[i,j]=np.log(v0/vgrid[kk])/(3*dlnr)
                    else:
                        kk=k
                        c[i,j] = 0.0
                    ima[i,j]=min(nbin-1,kk)
                    break
            c[j,i] = c[i,j]
            ima[j,i] = ima[i,j]
    return (c,ima)


In [5]:
def lin_interp(cck):
    n = len(cck[0])
    ck = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            jm = max(j-1,0)
            im = max(i-1,0)
            jp = min(j+1,n-1)
            ip = min(i+1,n-1)
            ck[i,j] = 0.125*(cck[i,jm]+cck[im,j]+cck[ip,j]+cck[i,jp])+0.5*cck[i,j]
            if i==j:
                ck[i,j]=0.5*ck[i,j]
    return ck

In [6]:
def g_init(x):
    return gamma_init(x,N0,k1,theta)

# kernel
def kernel(x,y):
    return golovin_kernel(x,y,B)

In [7]:
# define grid
vmin = emin / rhow
dlnr = np.log(2)/3/scal
gridax = 2**(1/scal)

vgrid = np.zeros(nbin)

vgrid[0] = vmin*0.5*(gridax+1)
for i in range(nbin-1):
    vgrid[i+1] = vgrid[i]*gridax
rgrid = (vgrid/4/np.pi*3)**(1/3)

# define initial condition
gin = np.zeros(nbin)
for i in range(nbin):
    gin[i] = g_init(vgrid[i])

# define the kernel
cck = np.zeros((nbin,nbin))
for i in range(nbin):
    for j in range(i,nbin):
        cck[i,j] = kernel(vgrid[i],vgrid[j])
        cck[j,i] = cck[i,j]
ck = lin_interp(cck)
ck = ck*dt*dlnr

# compute the courant numbers
(c,ima) = courant(vgrid,dlnr)

In [8]:
def coad(vgrid, gin, ima, ck, c, gmin=1e-60):
    g = gin.copy()
    nbin=len(vgrid)
    i0=0
    i1=nbin-1
    # find upper and lower integration limit
    for i in range(nbin):
        i0 = i
        if g[i] >= gmin:
            break
    for i in range(0,nbin-1,-1):
        i1 = i
        if g[i] >= gmin:
            break

    for i in range(i0,i1):
        for j in range(i,i1):
            k = int(ima[i,j])
            kp = k+1
            v0 = ck[i,j]*g[i]*g[j]
            v0 = min(v0, g[i]*vgrid[j])
            if j != k:
                v0 = min(v0,g[j]*vgrid[i])
            gsi=v0/vgrid[j]
            gsj=v0/vgrid[i]
            gsk=gsi+gsj
            g[i]=max(g[i]-gsi,0)
            g[j]=max(g[j]-gsj,0)
            gk=g[k]+gsk
            if gk > gmin:
                x1 = np.log(g[kp]/gk+1e-60)
                flux = gsk/x1*(np.exp(0.5*x1)-np.exp(x1*(0.5-c[i,j])))
                flux = min(min(flux,gk),gsk)
                
                g[k]=gk-flux
                g[kp]+=flux
    return g

In [9]:
gprev = gin.copy()
for ij in range(nt):
    gnext = coad(vgrid, gprev, ima, ck, c)
    gprev = gnext.copy()

In [10]:
vgrid

array([1.06370040e+00, 2.12740080e+00, 4.25480160e+00, 8.50960320e+00,
       1.70192064e+01, 3.40384128e+01, 6.80768256e+01, 1.36153651e+02,
       2.72307302e+02, 5.44614605e+02, 1.08922921e+03, 2.17845842e+03,
       4.35691684e+03, 8.71383368e+03, 1.74276674e+04, 3.48553347e+04,
       6.97106694e+04, 1.39421339e+05, 2.78842678e+05, 5.57685355e+05,
       1.11537071e+06, 2.23074142e+06, 4.46148284e+06, 8.92296569e+06,
       1.78459314e+07, 3.56918627e+07, 7.13837255e+07, 1.42767451e+08,
       2.85534902e+08, 5.71069804e+08, 1.14213961e+09, 2.28427922e+09])